### Импорты

In [ ]:
%pip install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.6/749.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 196.0/196.0 MB 57.0 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
from google.colab import drive
from ultralytics import YOLO, NAS
from PIL import Image

import os
import cv2
import glob
import yaml
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
import warnings

warnings.filterwarnings('ignore')
np.random.seed(12345)

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Работа/Experiments')

### Подготовка данных

In [ ]:
dataDir = 'void-dataset-yolo-obb_0_2_0/'
workingDir = os.getcwd()

In [ ]:
num_classes = 2
classes = ['void',
           'person']

In [ ]:
file_dict = {
    'train': os.path.join(dataDir, 'train/images'),
    'val': os.path.join(dataDir, 'validation/images'),
    'test': os.path.join(dataDir, 'validation/images'),
    'nc': num_classes,
    'names': classes
}

In [ ]:
# with open(os.path.join(workingDir, 'data.yaml'), 'w+') as f:
#     yaml.dump(file_dict, f)

In [ ]:
class_idx = {str(i): classes[i] for i in range(num_classes)}
class_stat = {}
data_len = {}

In [ ]:
for mode in ['train', 'validation']:
    class_count = {classes[i]: 0 for i in range(num_classes)}
    path = os.path.join(dataDir, mode, 'labels')

    for file in os.listdir(path):
        with open(os.path.join(path, file)) as f:
            lines = f.readlines()

            for cls in set([line[0] for line in lines]):
                class_count[class_idx[cls]] += 1

    data_len[mode] = len(os.listdir(path))
    class_stat[mode] = class_count

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5), sharey=True)

for i, mode in enumerate(['train', 'validation']):
    sns.barplot(pd.DataFrame({mode: class_stat[mode]}).T / data_len[mode] * 100, ax=ax[i])

    ax[i].set_title(mode)
    ax[i].tick_params(rotation=90)
    ax[i].set_ylabel('Percentage of classes')

plt.show()

In [ ]:
for mode in ['train', 'validation']:
    print(f'Image size in {mode} set:\n')
    image_size = 0

    for file in glob.glob(os.path.join(dataDir, mode, 'images', '*')):
        image = Image.open(file)

        if image.size != image_size:
            print(image.size)
            image_size = image.size

In [ ]:
for mode in ['train', 'validation']:
    files = glob.glob(os.path.join(dataDir, mode, 'images', '*'))

    print(f'{mode} set size: {len(files)}\n')

### Обучение

In [ ]:
model = YOLO('yolov8l-obb.pt')

In [ ]:
model.train(data=os.path.join(workingDir, 'void-dataset-yolo-obb_0_2_0/data.yaml'),
            task = 'detect',
            imgsz = 640,
            epochs = 20,
            batch = 16,
            mode = 'train',
            name = 'yolov8l_trained_model')

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8l-obb.pt, data=/content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8l_trained_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

train: Scanning /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/labels.cache... 510 images, 6 backgrounds, 1 corrupt: 100%|██████████| 510/510 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/images/2130706460#1$20240216145755477.jpg: ignoring corrupt image/label: negative label values [ -0.0022205]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/labels.cache... 155 images, 3 backgrounds, 1 corrupt: 100%|██████████| 155/155 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706450#1$20240208161333342.jpg: ignoring corrupt image/label: negative label values [ -0.0007185]


...

                   all        154       1735      0.797      0.704      0.757      0.235



20 epochs completed in 0.373 hours.
Optimizer stripped from runs/obb/yolov8l_trained_model/weights/last.pt, 89.4MB
Optimizer stripped from runs/obb/yolov8l_trained_model/weights/best.pt, 89.4MB

Validating runs/obb/yolov8l_trained_model/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-obb summary (fused): 287 layers, 44456601 parameters, 0 gradients, 168.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:20<00:00,  2.02s/it]


                   all        154       1735      0.781      0.725      0.764      0.236
                  void        154       1326      0.699      0.729      0.736       0.25
                person        154        409      0.862       0.72      0.791      0.222
Speed: 0.6ms preprocess, 18.7ms inference, 0.0ms loss, 26.6ms postprocess per image
Results saved to runs/obb/yolov8l_trained_model


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7eff64a9add0>
curves: []
curves_results: []
fitness: 0.28891523939602926
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.25039,     0.22195])
names: {0: 'void', 1: 'person'}
plot: True
results_dict: {'metrics/precision(B)': 0.7806523995673886, 'metrics/recall(B)': 0.7245946233633402, 'metrics/mAP50(B)': 0.7636204756007023, 'metrics/mAP50-95(B)': 0.23617021315106554, 'fitness': 0.28891523939602926}
save_dir: PosixPath('runs/obb/yolov8l_trained_model')
speed: {'preprocess': 0.5837539573768517, 'inference': 18.711316121088995, 'loss': 0.0010310829459846795, 'postprocess': 26.641625862616998}

In [ ]:
model = YOLO('yolov8m-obb.pt')

100%|██████████| 50.8M/50.8M [00:00<00:00, 65.9MB/s]


In [ ]:
model.train(data=os.path.join(workingDir, 'void-dataset-yolo-obb_0_2_0/data.yaml'),
            task = 'detect',
            imgsz = 640,
            epochs = 20,
            batch = 32,
            mode = 'train',
            name = 'yolov8m_trained_model')

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8m-obb.pt, data=/content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/data.yaml, epochs=20, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8m_trained_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

train: Scanning /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/labels.cache... 510 images, 6 backgrounds, 1 corrupt: 100%|██████████| 510/510 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/images/2130706460#1$20240216145755477.jpg: ignoring corrupt image/label: negative label values [ -0.0022205]


...

                   all        154       1735      0.834      0.695      0.751      0.231



20 epochs completed in 0.284 hours.
Optimizer stripped from runs/obb/yolov8m_trained_model/weights/last.pt, 53.4MB
Optimizer stripped from runs/obb/yolov8m_trained_model/weights/best.pt, 53.4MB

Validating runs/obb/yolov8m_trained_model/weights/best.pt...
Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m-obb summary (fused): 237 layers, 26401225 parameters, 0 gradients, 80.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:14<00:00,  2.82s/it]


                   all        154       1735      0.835      0.695      0.751      0.231
                  void        154       1326      0.743      0.655      0.697      0.237
                person        154        409      0.926      0.735      0.804      0.226
Speed: 0.3ms preprocess, 7.9ms inference, 0.0ms loss, 17.3ms postprocess per image
Results saved to runs/obb/yolov8m_trained_model


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dd0634e4e80>
curves: []
curves_results: []
fitness: 0.2832050777303419
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.23665,     0.22589])
names: {0: 'void', 1: 'person'}
plot: True
results_dict: {'metrics/precision(B)': 0.8345196063559337, 'metrics/recall(B)': 0.6952344868532125, 'metrics/mAP50(B)': 0.7506390094335769, 'metrics/mAP50-95(B)': 0.23126797420776027, 'fitness': 0.2832050777303419}
save_dir: PosixPath('runs/obb/yolov8m_trained_model')
speed: {'preprocess': 0.31251876385180977, 'inference': 7.863501449684044, 'loss': 0.000724544772854099, 'postprocess': 17.27254359753101}

In [ ]:
model = YOLO('yolov9c.pt')

In [ ]:
model.train(data=os.path.join(workingDir, 'void-dataset-yolo-obb_0_2_0/data.yaml'),
            task = 'detect',
            imgsz = 640,
            epochs = 20,
            batch = 16,
            mode = 'train',
            name = 'yolov9c_trained_model')

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov9c_trained_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

train: Scanning /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/labels.cache... 510 images, 6 backgrounds, 1 corrupt: 100%|██████████| 510/510 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/images/2130706460#1$20240216145755477.jpg: ignoring corrupt image/label: negative label values [ -0.0022205]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/labels.cache... 155 images, 3 backgrounds, 1 corrupt: 100%|██████████| 155/155 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706450#1$20240208161333342.jpg: ignoring corrupt image/label: negative label values [ -0.0007185]


...

                   all        154       1735      0.827      0.773      0.837      0.581



20 epochs completed in 0.480 hours.
Optimizer stripped from runs/detect/yolov9c_trained_model/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/yolov9c_trained_model/weights/best.pt, 51.6MB

Validating runs/detect/yolov9c_trained_model/weights/best.pt...
Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv9c summary (fused): 384 layers, 25320790 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:17<00:00,  3.52s/it]


                   all        154       1735      0.823      0.775      0.838      0.583
                  void        154       1326      0.722      0.715       0.76      0.465
                person        154        409      0.924      0.834      0.916        0.7
Speed: 0.5ms preprocess, 34.3ms inference, 0.0ms loss, 16.0ms postprocess per image
Results saved to runs/detect/yolov9c_trained_model


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dd065af2590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
model = YOLO('yolov9c.pt')

In [ ]:
model.train(data=os.path.join(workingDir, 'void-dataset-yolo-obb_0_2_0/data.yaml'),
            task = 'test',
            imgsz = 640,
            epochs = 20,
            batch = 16,
            mode = 'train',
            name = 'yolov9c_trained_2_model')

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=test, mode=train, model=yolov9c.pt, data=/content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov9c_trained_2_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

train: Scanning /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/labels.cache... 510 images, 6 backgrounds, 1 corrupt: 100%|██████████| 510/510 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/train/images/2130706460#1$20240216145755477.jpg: ignoring corrupt image/label: negative label values [ -0.0022205]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/My Drive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/labels.cache... 155 images, 3 backgrounds, 1 corrupt: 100%|██████████| 155/155 [00:00<?, ?it/s]

...

      14/20      11.2G      1.196     0.8827      1.077         97        640: 100%|██████████| 32/32 [00:49<00:00,  1.54s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.32s/it]

                   all        154       1735       0.78      0.666      0.752      0.486



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/20      11.1G      1.141     0.8217      1.044         85        640: 100%|██████████| 32/32 [00:47<00:00,  1.49s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:08<00:00,  1.62s/it]

                   all        154       1735      0.805      0.713      0.804      0.532


### Тест

In [ ]:
model = YOLO('runs/obb/yolov8l_trained_model/weights/best.pt')

In [ ]:
results = model.predict(source=os.path.join(dataDir, 'validation', 'images'), save=True, show_conf=False, show_labels=False)

In [ ]:
model = YOLO('runs/obb/yolov8m_trained_model/weights/best.pt')

In [ ]:
results = model.predict(source=os.path.join(dataDir, 'validation', 'images'), save=True, show_conf=False, show_labels=False)


image 1/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/05 утро.jpg: 384x640 108.1ms
image 2/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240202170425597.jpg: 384x640 26.5ms
image 3/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240207131554395.jpg: 384x640 26.5ms
image 4/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240208133819084.jpg: 384x640 26.6ms
image 5/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240209135630344.jpg: 384x640 26.5ms
image 6/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240209135638649.jpg: 384x640 26.5ms
image 7/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20

In [ ]:
model = YOLO('runs/detect/yolov9c_trained_model/weights/best.pt')

In [ ]:
results = model.predict(source=os.path.join(dataDir, 'validation', 'images'), save=True, show_conf=False, show_labels=False)


image 1/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/05 утро.jpg: 384x640 6 voids, 3 persons, 165.7ms
image 2/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240202170425597.jpg: 384x640 12 voids, 1 person, 31.9ms
image 3/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240207131554395.jpg: 384x640 12 voids, 33.0ms
image 4/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240208133819084.jpg: 384x640 6 voids, 32.5ms
image 5/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240209135630344.jpg: 384x640 8 voids, 31.8ms
image 6/155 /content/drive/MyDrive/Работа/Experiments/void-dataset-yolo-obb_0_2_0/validation/images/2130706433#1$20240209135638649.jpg: 384x640 10 voids, 31.8ms
image 7/155 /content/drive/MyDrive/Ра

In [ ]:
# pred = glob.glob('runs/obb/predict/*')
# n = 5
# num_rows = len(pred) // 5

In [ ]:
# fig, axs = plt.subplots(num_rows, n, figsize=(25, 5 * num_rows))

# for i in range(num_rows):
#     for j in range(n):
#         idx = i * n + j

#         if idx < len(pred):
#             image = Image.open(pred[idx])
#             axs[i, j].imshow(image)
#             axs[i, j].grid(False)
#             axs[i, j].axis('off')

# plt.show()